# Objetivos
- Sacar datos gasolina por mes (2019-2022) : https://energia.serviciosmin.gob.es/shpCarburantes/ (SELENIUM)
    - Lo vamos a sacar de Gasolina 95,98,Diesel y Diesel Premium

- Sacar datos Agua:
    - https://facua.org/es/tablas/agua2019.pdf
    - https://facua.org/es/tablas/agua_2020.pdf
    - https://facua.org/es/tablas/agua2021.pdf
    - https://facua.org/es/tablas/agua2022.pdf

In [ ]:
# Importamos las librerías que necesitamos

# Librerías para control y automatización de navegadores (Web Scraping)
# -----------------------------------------------------------------------
from selenium import webdriver                   # Controla el navegador automáticamente para realizar tareas de scraping
from selenium.webdriver.chrome.service import Service  # Gestiona los servicios de ChromeDriver para ejecutar Chrome sin intervención manual
from selenium.webdriver.common.by import By      # Selecciona y localiza elementos en el DOM de una página web
from selenium.webdriver.chrome.options import Options  # Configura opciones de Chrome, como el modo sin interfaz (headless)
from selenium.webdriver.support.ui import WebDriverWait  # Define tiempos de espera para asegurar que elementos estén presentes en el DOM
from selenium.webdriver.common.keys import Keys  # Simula la pulsación de teclas en el navegador
from selenium.webdriver.support import expected_conditions as EC  # Establece condiciones para determinar cuándo ciertos elementos están presentes

# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Manipula y analiza datos en estructuras DataFrame
import numpy as np              # Realiza cálculos numéricos y manipulación de arrays multidimensionales
import datetime                 # Maneja fechas y horas, útil para organizar datos temporalmente

# Librerías del sistema y utilidades
# -----------------------------------------------------------------------
import os                       # Interactúa con el sistema operativo, gestionando rutas y directorios
import sys                      # Permite modificar rutas de búsqueda de módulos y gestionar la ejecución del programa
sys.path.append("../")          # Añade el directorio padre al PATH para permitir la importación de módulos personalizados

# Importación de funciones personalizadas para scraping y procesamiento de datos
from src.SupportScraping import (  
    precios_combustible,        # Función para realizar scraping de precios de combustible
    excel_a_Dataframe,          # Función para procesar archivos Excel y convertirlos en DataFrames
    pdf_a_df                    # Función para extraer tablas de archivos PDF y convertirlas en DataFrames
)

from time import sleep          # Pausa la ejecución del programa por un tiempo determinado
import shutil                   # Permite realizar operaciones de alto nivel en archivos y colecciones de archivos, como copiar y mover

# Obtener precios Gasolina

## Generar rangos de fecha

Para poder obtener precios de la Gasolina, necesitamos entrar a la web en rangos de fecha de X a Y

In [2]:
fechas_inicio_mes = pd.date_range(start='2019-01-01', end='2022-12-01', freq='YS')
fechas_fin_mes = pd.date_range(start='2019-01-31', end='2022-12-31', freq='YE')
# Lo pasamos a lista para nuestra función
lista_inicio_mes = fechas_inicio_mes.strftime('%d/%m/%Y').tolist()
lista_fin_mes = fechas_fin_mes.strftime('%d/%m/%Y').tolist()

## Descargar Datos Combustible

In [3]:
precios_combustible(lista_inicio_mes,lista_fin_mes)

## Preparar DataFrame
Dado que es un excel necesitamos la extension xlrd

pip install xlrd

Ahora con la función le pasamos la ruta absoluta donde están nuestros .xls para que nos devuelva un solo DataFrame

In [2]:
ruta_padre = os.path.dirname(os.getcwd())
ruta_archivos = os.path.join(ruta_padre, "datos", "03_PreciosCombustible", "DatosDescargados")

In [3]:
df_combustible = excel_a_Dataframe(ruta_archivos)

In [6]:
df_combustible.head()

,Fecha,Precio,combustible
0,01/2019,1.228664,Gasolina 95
1,02/2019,1.271043,Gasolina 95
2,03/2019,1.305643,Gasolina 95
3,04/2019,1.369528,Gasolina 95
4,05/2019,1.387684,Gasolina 95


### Ponemos la fecha en DateTime

In [8]:
df_combustible["Fecha"] = pd.to_datetime(df_combustible["Fecha"],format="%Y-%m-%d")

In [10]:
df_combustible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Fecha        192 non-null    datetime64[ns]
 1   Precio       192 non-null    float64       
 2   combustible  192 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.6+ KB


### Guardamos y pasamos a los precios del Agua

In [11]:
df_combustible.to_csv("../datos/03_PreciosCombustible/01_TablaPreciosCombustible.csv")

# Precios Agua
Vamos a leer varios PDF y vamos a quedarnos con el precio del agua en la comunidad de Madrid

pip install pdfplumber

In [2]:
ruta_padre = os.path.dirname(os.getcwd())
ruta_hacer = os.path.join(ruta_padre, "datos", "04_PreciosAgua","01_PDF")
pdf_a_df(ruta_hacer)

agua2019.pdf
agua2020.pdf
agua2021.pdf
agua2022.pdf
agua2022.pdf no se ha podido realizar


### 2022 Falla
Esto se debe a que Facua ha puesto las tablas como una imagen, se ha intentado leer sin éxito, así que vamos a sacar la información del INE
- https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176834&menu=resultados&idp=1254735976602

In [3]:
ruta_padre = os.path.dirname(os.getcwd())
ruta_hacer = os.path.join(ruta_padre, "datos", "04_PreciosAgua", "DatosDescargados")
os.makedirs(ruta_hacer,exist_ok=True)
# lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
chrome_options = webdriver.ChromeOptions()
# establacemos las preferencias que queremos
prefs = {               
"download.default_directory": ruta_hacer,  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
"download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
"directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}
url = "https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176834&menu=resultados&idp=1254735976602"
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.find_element("xpath","/html/body/div[1]/main/section[2]/div/div/div[1]/ul/li/ul/li[2]/a").click()
sleep(0.3)
driver.find_element("xpath","/html/body/div[1]/main/div[2]/ul/li[3]/ul/li[3]/a").click()
sleep(0.3)
tabla1 = driver.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div")
sleep(1)
tabla1.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[2]/ul/li[1]/ul/li[1]/label").click()
sleep(0.5)
tabla1.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[2]/ul/li[1]/ul/li[2]/label").click()
sleep(1)
tabla1.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[1]/input").send_keys("Ceuta")
tabla1.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[1]/button").click()
sleep(0.3)
tabla1.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[1]/div/fieldset/div[2]/ul/li[2]/ul/li[1]/ul/li[13]/node/label").click()
sleep(0.3)
driver.find_element("xpath","/html/body/div[1]/main/form/div[5]/input[3]").click()
sleep(0.5)
driver.find_element("xpath","/html/body/div[1]/main/ul/li/div/div/form[2]/button").click()
select_download = WebDriverWait(driver,40).until(EC.presence_of_element_located(("xpath","/html/body/div[7]/article/div/div/iframe")))
driver.switch_to.frame(select_download)
driver.find_element("xpath","/html/body/form/ul/li[4]/label").click()
sleep(2)
driver.quit()

### Movemos y renombramos

In [4]:
old_name= os.listdir(ruta_hacer)[0]
new_name = "Precio2022Madrid.csv"
os.rename(f"{ruta_hacer}/{old_name}",f"{ruta_hacer}/{new_name}")
old_name = os.listdir(ruta_hacer)[0]
old_path = os.path.join(ruta_hacer, old_name)
ruta_directorio_anterior = os.path.dirname(ruta_hacer)
new_path = os.path.join(ruta_directorio_anterior, "PreciosAgua2022.csv")
shutil.copy2(old_path, new_path)
print(f"Archivo copiado y renombrado a: {new_path}")


Archivo copiado y renombrado a: c:\Users\ruipe\Desktop\Ejercicios\Semana 5\Proyecto5-Variacion-Precios-Luz-Agua-Gas-Combustible-PreCovid-PostCovid-Madrid\datos\04_PreciosAgua\PreciosAgua2022.csv


# Limpiar DF y concatenar


In [5]:
df2019 = pd.read_csv("../datos/04_PreciosAgua/PreciosAgua2019.csv",index_col="Unnamed: 0")
df2020 = pd.read_csv("../datos/04_PreciosAgua/PreciosAgua2020.csv",index_col="Unnamed: 0")
df2021 = pd.read_csv("../datos/04_PreciosAgua/PreciosAgua2021.csv",index_col="Unnamed: 0")
df2022 = pd.read_csv("../datos/04_PreciosAgua/PreciosAgua2022.csv",encoding="latin-1",sep=";")

In [6]:
df2019 = df2019[["Ciudades / Total\nmensual","Consumo de\n10m3 con\ncontador\nindividual de\n13mm"]]
cond = df2019["Ciudades / Total\nmensual"].str.contains("Madrid",case=False,na=False)
df2019 = df2019[cond].reset_index(drop=True)
df2019 = df2019.rename(columns={"Ciudades / Total\nmensual":"ciudad",
                                "Consumo de\n10m3 con\ncontador\nindividual de\n13mm":"consumo de 10m3"})
df2019["ciudad"] = "Madrid"
df2019["año"] = 2019
df2019["consumo de 10m3"] = df2019["consumo de 10m3"].str.replace(",",".").str.strip(" €").astype(float)
df2019["consumo (€/m3)"] = round(df2019["consumo de 10m3"]/10,2)
df2019 = df2019[["ciudad","consumo (€/m3)","año"]]
df2019

,ciudad,consumo (€/m3),año
0,Madrid,1.57,2019


In [7]:
df2020 = df2020[["Ciudades / Total mensual","Consumo\nde 9m3 con\ncontador\nindividual\nde 13mm"]]
cond = df2020["Ciudades / Total mensual"].str.contains("Madrid",case=False,na=False)
df2020 = df2020[cond].reset_index(drop=True)
df2020 = df2020.rename(columns={"Ciudades / Total mensual":"ciudad",
                                "Consumo\nde 9m3 con\ncontador\nindividual\nde 13mm":"consumo de 9m3"})
df2020["ciudad"] = "Madrid"
df2020["año"] = 2020
df2020["consumo de 9m3"] = df2020["consumo de 9m3"].str.replace(",",".").str.strip(" €").astype(float)
df2020["consumo (€/m3)"] = round(df2020["consumo de 9m3"]/9,2)
df2020 = df2020[["ciudad","consumo (€/m3)","año"]]
df2020

,ciudad,consumo (€/m3),año
0,Madrid,1.65,2020


In [8]:
df2021 = df2021[["Ciudades / Total\nmensual","Consumo\nde 9m3 con\ncontador\nindividual\nde 13mm"]]
cond = df2021["Ciudades / Total\nmensual"].str.contains("Madrid",case=False,na=False)
df2021 = df2021[cond].reset_index(drop=True)
df2021 = df2021.rename(columns={"Ciudades / Total\nmensual":"ciudad",
                                "Consumo\nde 9m3 con\ncontador\nindividual\nde 13mm":"consumo de 9m3"})
df2021["ciudad"] = "Madrid"
df2021["año"] = 2021
df2021["consumo de 9m3"] = df2021["consumo de 9m3"].str.replace(",",".").str.strip(" €").astype(float)
df2021["consumo (€/m3)"] = round(df2021["consumo de 9m3"]/9,2)
df2021 = df2021[["ciudad","consumo (€/m3)","año"]]
df2021

,ciudad,consumo (€/m3),año
0,Madrid,1.65,2021


In [9]:
df2022 = df2022[["Comunidades y Ciudades Autónomas","Total","año"]]
df2022 = df2022.rename(columns={"Comunidades y Ciudades Autónomas":"ciudad",
                        "Total":"consumo (€/m3)"})
df2022["consumo (€/m3)"] = df2022["consumo (€/m3)"].str.replace(",",".").astype(float)
df2022["ciudad"] = "Madrid"
df2022

,ciudad,consumo (€/m3),año
0,Madrid,1.8,2022


In [10]:
df_agua = pd.concat([df2019,df2020,df2021,df2022],ignore_index=True)
df_agua

,ciudad,consumo (€/m3),año
0,Madrid,1.57,2019
1,Madrid,1.65,2020
2,Madrid,1.65,2021
3,Madrid,1.80,2022


# Creamos un precio por mes

Dado que no hay datos mensuales sobre el agua, con objetivo de mantener la estructura de datos, vamos a asumir que el precio del agua varia cada año y se mantiene cada mes

In [11]:
df_agua = df_agua.drop(columns=["ciudad"])
years = df_agua['año'].tolist()
precios = df_agua['consumo (€/m3)'].tolist()
fechas = []
precios_mes = []
for year, price in zip(years, precios):
    fechas.extend(pd.date_range(start=f'{year}-01-01', periods=12, freq='MS'))
    precios_mes.extend([price] * 12)
    
df_agua = pd.DataFrame({
    'fecha': fechas,
    'consumo (€/m3)': precios_mes
})

In [13]:
df_agua.head()

,fecha,consumo (€/m3)
0,2019-01-01,1.57
1,2019-02-01,1.57
2,2019-03-01,1.57
3,2019-04-01,1.57
4,2019-05-01,1.57


In [14]:
df_agua.to_csv("../datos/04_PreciosAgua/01_PreciosAguaFinal.csv")